<a href="https://colab.research.google.com/github/Shujaat123/AntiOxiLSE/blob/main/AntiOxiPred_LSE_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AntiOxi-LSE: Prediction of Antioxidant Proteins Using Latent Space Encoding of Composition of k-Spaced Amino Acid Pairs**


This code provide python implementation of AntiOxi-LSE algorithm.

# Loading Useful packages

In [1]:
## Load useful packages
import sys, os, re, gc
from scipy.io import savemat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils.np_utils import to_categorical
from collections import Counter
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras import optimizers
from keras import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Feature-Extraction

The CKSAAP feature encoding calculates the frequency of amino acid pairs separated by any k residues. The CKSAAP encoding scheme reflects the amino acid pair information in small and large range with in the peptides depending upon the value of k(gap). The encoding scheme is utilized from iFeature web server, using the following download link: (https://github.com/Superzchen/iFeature).


In [2]:
## Define CKSAAP feature-extraction function
def Convert_Seq2CKSAAP(train_seq, gap=8):
  cksaapfea = []
  seq_label = []
  for sseq in train_seq:
    temp= CKSAAP([sseq], gap=8)
    cksaapfea.append(temp[1][1:])
    seq_label.append(sseq[0])

  x = np.array(cksaapfea)
  y = np.array(seq_label)
  y[y=='POS']=1
  y[y=='NEG']=0
  y = to_categorical(y)

  return x,y

def minSequenceLength(fastas):
	minLen = 10000
	for i in fastas:
		if minLen > len(i[1]):
			minLen = len(i[1])
	return minLen

def CKSAAP(fastas, gap=5, **kw):
	if gap < 0:
		print('Error: the gap should be equal or greater than zero' + '\n\n')
		return 0

	if minSequenceLength(fastas) < gap+2:
		print('Error: all the sequence length should be larger than the (gap value) + 2 = ' + str(gap+2) + '\n' + 'Current sequence length ='  + str(minSequenceLength(fastas)) + '\n\n')
		return 0

	AA = 'ACDEFGHIKLMNPQRSTVWY'
	encodings = []
	aaPairs = []
	for aa1 in AA:
		for aa2 in AA:
			aaPairs.append(aa1 + aa2)
	header = ['#']
	for g in range(gap+1):
		for aa in aaPairs:
			header.append(aa + '.gap' + str(g))
	encodings.append(header)
	for i in fastas:
		name, sequence = i[0], i[1]
		code = [name]
		for g in range(gap+1):
			myDict = {}
			for pair in aaPairs:
				myDict[pair] = 0
			sum = 0
			for index1 in range(len(sequence)):
				index2 = index1 + g + 1
				if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[index2] in AA:
					myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
					sum = sum + 1
			for pair in aaPairs:
				code.append(myDict[pair] / sum)
		encodings.append(code)
	return encodings


In [3]:
def load_seq_data(data_path):
  dataset = pd.read_csv(data_path,names=None,index_col=0, header=None)
  seq = []
  POS_sample_count = 0
  NEG_sample_count = 0
  for row in dataset.iterrows():
    if(row[0].startswith('ind_pos')):
      POS_sample_count = POS_sample_count +1
      label = 'POS'
      array = [label, np.asarray(row[1])[0]]
      name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
      seq.append([name, sequence])
    
    elif(row[0].startswith('all_neg')):
      NEG_sample_count = NEG_sample_count +1
      label = 'NEG'
      array = [label, np.asarray(row[1])[0]]
      name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
      seq.append([name, sequence])

    elif(row[0].startswith('>anti_')):
      POS_sample_count = POS_sample_count +1
      label = 'POS'
      continue  
    elif(row[0].startswith('>nonanti_')):
      NEG_sample_count = NEG_sample_count +1
      label = 'NEG'
      continue
    else:
      array = [label, row[0]]
      name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
      seq.append([name, sequence])
  print('# of POS samples',POS_sample_count,'\t # of NEG samples',NEG_sample_count)
  return seq

## Loading and pre-processing protein's dataset

In [4]:
# ## Loading and pre-processing PVP prediction dataset
pos_seq_path = 'https://raw.githubusercontent.com/Shujaat123/AntiOxiLSE/main/AodPred/anti.txt'
neg_seq_path = 'https://raw.githubusercontent.com/Shujaat123/AntiOxiLSE/main/AodPred/nonanti.txt'
pos_all_seq = load_seq_data(pos_seq_path)
neg_all_seq = load_seq_data(neg_seq_path)
ALL_seq = pos_all_seq + neg_all_seq

print(len(pos_all_seq), len(neg_all_seq), len(ALL_seq))



# of POS samples 253 	 # of NEG samples 0
# of POS samples 0 	 # of NEG samples 1552
253 1552 1805


# Designing an Auto-Encoder-based classifier model

In [5]:
## Designing an Auto-Encoder-Classifier model
def LSE_Final_Model(input_shape=3600, LV=5):
    # Encoder Network
    enc_input = Input(shape=(input_shape,), name='enc_input')
    enc_l1 = Dense(50, activation='relu', name='encoder_layer1')(enc_input)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Dropout(rate = 0.3)(enc_l1)

    enc_l2 = Dense(25, activation='relu', name='encoder_layer2')(enc_l1)
    enc_l2 = BatchNormalization()(enc_l2)
    enc_l2 = Dropout(rate = 0.3)(enc_l2)

    enc_l3 = Dense(10, activation='relu', name='encoder_layer3')(enc_l2)
    enc_l3 = BatchNormalization()(enc_l3)
    enc_l3 = Dropout(rate = 0.3)(enc_l3)

    encoder_output = Dense(LV, activation='sigmoid', name='encoder_output')(enc_l3)
  
    # Classifier Network
    class_l1 = Dense(10, activation='relu', name='class_layer1')(encoder_output)
    class_l2 = Dense(10, activation='relu', name='class_layer3')(class_l1)
    class_output = Dense(2, activation='softmax', name='class_output')(class_l2)

    # Decoder Network
    dec_l1 = Dense(10, activation='relu', name='decoder_layer1')(encoder_output)
    dec_l1 = BatchNormalization()(dec_l1)
    dec_l1 = Dropout(rate = 0.3)(dec_l1)

    dec_l2 = Dense(25, activation='relu', name='decoder_layer2')(dec_l1)
    dec_l2 = BatchNormalization()(dec_l2)
    dec_l2 = Dropout(rate = 0.3)(dec_l2)

    dec_l3 = Dense(50, activation='relu', name='decoder_layer3')(dec_l2)
    dec_l3 = BatchNormalization()(dec_l3)
    dec_l3 = Dropout(rate = 0.3)(dec_l3)

    decoder_output = Dense(input_shape, activation='sigmoid', name='decoder_output')(dec_l3)

    model = Model(inputs=[enc_input], outputs=[class_output, decoder_output])

    # Compiling model
    model.compile(optimizer='rmsprop',
                  loss={'class_output': 'binary_crossentropy', 'decoder_output': 'mean_squared_error'},
                  loss_weights={'class_output': 0.01, 'decoder_output': 0.99},
                  metrics=[metrics.categorical_accuracy])
    # Here I used rmsprops optimizer with default values, two objective functions are optimized
    # using  weight factors [1 for classifier and 0.1 for decoder loss]
    return model

## Define performance measures

In [6]:
## Define performance measures
def yoden_index(y, y_pred):
  tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
  j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
  return j

def pmeasure(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn )
    specificity = tn / (tn + fp)
    f1score = (2 * tp) / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,Stats):
  print(msg.upper())
  print(70*'-')
  print('Accuracy:',Stats[0])
  print('Sensitivity:',Stats[1])
  print('Specificity:',Stats[2])
  print('F1-Score:',Stats[3])
  print('MCC:',Stats[4])
  print('Balance Accuracy:',Stats[5])
  print('Youden-Index:',Stats[6])
  print('AUC:',Stats[7])
  print('AUPR:',Stats[8])
  print('Reconstruction MSE:',Stats[9])
  print(70*'-')

def Calculate_Stats(y_actual,y_pred, y_score):
  acc = accuracy_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  sen = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['Sensitivity']
  spe = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['Specificity']
  f1 = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['F1-Score']
  mcc = matthews_corrcoef(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  bacc = balanced_accuracy_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  yi = yoden_index(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  
  pre, rec, _ = precision_recall_curve(y_actual.argmax(axis=1), y_score, pos_label=1)
  fpr, tpr, _ = roc_curve(y_actual.argmax(axis=1), y_score, pos_label=1)
  auroc = auc(fpr, tpr)
  aupr = auc(rec, pre)
    
  return acc, sen, spe, f1, mcc, bacc, yi, auroc, aupr  

def label_by_th(y_pred, threshold=0.5):
  y_pred_copy = y_pred.copy()
  y_pred_copy[y_pred>= threshold] = 1 
  y_pred_copy[y_pred<threshold] = 0 
  return y_pred_copy

def cutoff_youdens_j(fpr,tpr,thresholds):
  j_scores = tpr-fpr
  j_ordered = sorted(zip(j_scores,thresholds))
  return j_ordered[-1][1]

  ## Perform Monte-Carlos Simulations for [num_Trials]\# of independent Trials


In [ ]:
  ## Perform Monte-Carlos Simulations for [num_Trials]# of independent Trials
LVs = range(6,7)
num_Trails = 10
gaps = range(6,7)

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

for gap_ind in range(0,len(gaps)):
    [DataX, LabelY] = Convert_Seq2CKSAAP(ALL_seq, gap=gaps[gap_ind])
    plist = list(np.asarray(np.where(LabelY[:,1]==1)).flatten())
    nlist = list(np.asarray(np.where(LabelY[:,1]==0)).flatten())
    total_list = plist + nlist

    for lv_ind in range(0,len(LVs)):
      Stats =[]
      for loop_ind in range(0,num_Trails):
        ## train
        p_train = sample(plist, 200) # out of 253
        n_train = sample(nlist, 1240) # out of 1552
        train_list = p_train + n_train
        X_train = DataX[list(train_list)]
        y_train = LabelY[list(train_list)]

        ## test
        test_list = set(total_list) - (set(train_list))
        X_test = DataX[list(test_list)]
        y_test = LabelY[list(test_list)]
        
        model = LSE_Final_Model(input_shape = X_train.shape[1],LV=LVs[lv_ind])
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=100)
        checkpoint = ModelCheckpoint('models\\model-best.h5',
                                    verbose=0, monitor='val_loss',save_best_only=True, mode='auto')
        history = model.fit({'enc_input': X_train},
                          {'class_output': y_train, 'decoder_output': X_train},
                          validation_data = ({'enc_input': X_test},
                          {'class_output': y_test, 'decoder_output': X_test}),
                          epochs=1000, batch_size=y_train.shape[0], callbacks=[checkpoint, es], verbose=0)
        del model  # deletes the existing model
        model = load_model('models\\model-best.h5')

        y_train_pred, X_train_pred = model.predict(X_train,batch_size=1800, verbose=0)
        y_train_score = y_train_pred[:,1]
        # y_train_pred = to_categorical(y_train_pred.argmax(axis=1))
        MSE_X_train_pred = (np.square(X_train_pred - X_train)).mean(axis=1)

        y_test_pred, X_test_pred = model.predict(X_test,batch_size=200, verbose=0)
        y_test_score = y_test_pred[:,1]
        
        # Optimal Threshold
        fpr, tpr, thresholds_AUC = roc_curve(y_test.argmax(axis=1), y_test_score)
        precision, recall, thresholds_AUPR = precision_recall_curve(y_test.argmax(axis=1),y_test_score)

        ## Optimal Threshold metrics
        distance = (1-fpr)**2+(1-tpr)**2
        EERs = (1-recall)/(1-precision)
        positive = sum(y_test.argmax(axis=1))
        negative = y_test.shape[0]-positive
        ratio = negative/positive
        opt_t_AUC = thresholds_AUC[np.argmin(distance)]
        opt_t_AUPR = thresholds_AUPR[np.argmin(np.abs(EERs-ratio))]
        opt_yodens_j = cutoff_youdens_j(fpr, tpr, thresholds_AUC)        
        y_test_pred_th = label_by_th(y_test_score, opt_yodens_j)
        y_test_pred = to_categorical(y_test_pred_th)
        MSE_X_test_pred = (np.square(X_test_pred - X_test)).mean(axis=1)

        # re-scaled training pred label
        y_train_pred_th = label_by_th(y_train_score, opt_yodens_j)
        y_train_pred = to_categorical(y_train_pred_th)

        print(confusion_matrix(y_test.argmax(axis=1), y_test_pred.argmax(axis=1), labels=[0,1]).ravel())

        ## Training Measures
        tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr = Calculate_Stats(y_train,y_train_pred, y_train_score);
        
        ## Test Measures
        t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_test,y_test_pred, y_test_score);

        Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr, -10*np.log10(MSE_X_train_pred.mean()),
                      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr,-10*np.log10(MSE_X_test_pred.mean())])
        print('CKSAAP-Gap:',gaps[gap_ind], 'LV=',LVs[lv_ind],'Trial:',loop_ind,
              ' \nTraining/ Test Youden-index:', tr_yi,'/',t_yi,
              ' \nTraining/ Test MCC:', tr_mcc,'/',t_mcc,
              ' \nTraining/ Test AUC:', tr_auc,'/',t_auc,
              ' \nTraining/ Test AUPR:', tr_aupr,'/',t_aupr,
              ' \nTraining/ Test MSE (dB):', -10*np.log10(MSE_X_train_pred.mean()), '/', -10*np.log10(MSE_X_test_pred.mean()))
      
      del model  # deletes the existing model
      Statistics = np.asarray(Stats)
      filename = 'AntiOxi_LSE_STATS_CKSAAP_GAP' + str(gaps[gap_ind]) + '_LV' + str(LVs[lv_ind]) + '.mat'
      savemat(filename,{'Statistics':Statistics})
      print('SAVING... '+ filename)

[253  59  18  35]
CKSAAP-Gap: 6 LV= 6 Trial: 0  
Training/ Test Youden-index: 0.7682258064516128 / 0.4712747943880018  
Training/ Test MCC: 0.5778612163929879 / 0.3797002404763455  
Training/ Test AUC: 0.9669072580645162 / 0.7879172714078374  
Training/ Test AUPR: 0.8541101019034921 / 0.5639361478387012  
Training/ Test MSE (dB): 36.86782363884182 / 36.99300289728028


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in true_divide


[260  52  14  39]
CKSAAP-Gap: 6 LV= 6 Trial: 1  
Training/ Test Youden-index: 0.8306451612903225 / 0.5691823899371069  
Training/ Test MCC: 0.6777854358675696 / 0.4635223814212741  
Training/ Test AUC: 0.970391129032258 / 0.8437348814707305  
Training/ Test AUPR: 0.805802971558433 / 0.5828729100535703  
Training/ Test MSE (dB): 42.40571907072861 / 42.32322874851227


## Show Classification/Reconstruction Statistics

In [ ]:
## Show Classification/Reconstruction Statistics
Show_Statistics('Training Results (MEAN)',Statistics.mean(axis=0)[0:10])
Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[10:20])